# Memory management in Rust 🦀🦀


[Pierre-Antoine Champin](http://champin.net/)

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/2.0/fr/"><img alt="Contrat Creative Commons" style="border-width:0" src="http://i.creativecommons.org/l/by-nc-sa/2.0/fr/88x31.png" /></a>

## Background: the stack and the heap

The stack

- contains the data of variables,
- whose size is known at compile-time (static),
- managed in a *last in, first out* fashion by the compiler (in all languages).

The heap

- contains data whose size is known at runtime (dynamic),
- allocated/freed in a less predictable manner,
- managed differently depending on the language.

### Heap management

In C
- the developer is in charge of freeing memory,
- which is error-prone (memory freed too soon, too late, never...).

In Java
- the garbage-collector is in charge of freeing memory,
- which can lead to performance issues.

### Heap management

In Rust

- the **compiler** is in charge of freeing memory.
- The developer must provide enough information to make this possible,
- which is achieved with the concepts of **ownership** and **borrowing**. 🦀🦀

## Move semantics

In most programming languages,
assignment has *copy* semantics.

```c
v1 = init();
v2 = v1;
// v1 et v2 contain "the same thing"
```

In Rust, assignment has *move* semantics:

* conceptually, data is *moved* from `v1` to `vZ`;
* after assignment, `v1` is considered unitialized.

### Example: the `Vec` type

In [2]:
let v1: Vec<i32> = vec![22, 44, 66];

<div style="text-align: center">
<img alt="A Vec in memory" src="images/trpl04-01.svg" style="display: inline; width: 12em;">
</div>

### With the copy semantics (C)

```c
Vec v1 = init_vec();
Vec v2 = v1;
```

<div style="text-align: center"> <img alt="Shallow copy" src="images/trpl04-02.svg" style="display: inline; width: 12em;"> </div>

### With the move semantics (Rust) 🦀

In [3]:
let v1: Vec<i32> = vec![22, 44, 66];
println!("v1={:?}", &v1);
let v2 = v1;
println!("v2={:?}", &v2);
//println!("v1={:?}", &v1); // DOES NOT COMPILE

v1=[22, 44, 66]
v2=[22, 44, 66]


<div style="text-align: center">
<img alt="After a move assignment" src="images/trpl04-04.svg" style="display: inline; width: 12em;">
</div>

### Explicit (deep) copy

In [4]:
let v1: Vec<i32> = vec![22, 44, 66];
let v2 = v1.clone(); // explicit (deep) copy
println!("v1={:?} v2={:?}", &v1, &v2);

v1=[22, 44, 66] v2=[22, 44, 66]


<div style="text-align: center">
<img alt="After a deep copy" src="images/trpl04-03.svg" style="display: inline; width: 12em;">
</div>

### Implicit copy

While the move semantics is the rule,
there are exceptions.

In particular, assignments of all base types (ints, floats, booleans...)
have the copy semantics,
because the compiler knows that this copy is *safe*.

In [5]:
let x = 42;
let y = x; // copy instead of move
println!("x={} y={}", x, y);

x=42 y=42


More generally,
any type implementing the [`Copy` trait](https://doc.rust-lang.org/stable/std/marker/trait.Copy.html)
uses the copy semantics for assignment.

User-defined types can be made to implement the `Copy` trait,
but this is not always possible or appropriate (e.g. the `Vec` type).

## Ownership 🦀🦀

* Any piece of data is **owned** by exactly one variable (or field in a structure).

* Conceptually, assigning a variable to another one *moves* the data,
  and **transfers** the ownership to the destination variable.
  
* When a variable is no longer used (e.g. when exiting its scope)
  if it still owns some data, these data are **freed**.

<div style="text-align: center">
<img alt="A Vec in memory" src="images/trpl04-01.svg" style="display: inline; width: 12em;">
</div>

In [6]:
fn fibo(nb: usize) -> Vec<i32> {
    let mut v1 = vec![];        // 1. The Vec is created here
    for i in 0..nb {
        if i < 2 { v1.push(1); }
        else     { v1.push(v1[i-2] + v1[i-1]); }
    }
    v1                          // 2. Its ownership is transfered to the caller
}

fn sum(v2: Vec<i32>) -> i32 {
    v2.into_iter().sum()
}                               // 5. We reach the end of `v2`'s scope;
                                //    it still owns the Vec, so the Vec is freed

{ 
    let n = 7;
    let mut v3 = fibo(n);       // 3. The Vec created by `fibo` is moved in `v3`
    println!("v3={:?}", &v3);
    let s = sum(v3);            // 4. The Vec is moved into `sum`
    println!("the sum of the {} first Fibonacci numbers is {}", n, s);
    // println!("{:?}", v3); // DOES NOT COMPILE
};

v3=[1, 1, 2, 3, 5, 8, 13]
the sum of the 7 first Fibonacci numbers is 33


## Borrow 🦀🦀

In [7]:
let v1: Vec<i32> = vec![22, 44, 66];
{
    let v2 = &v1;
    println!("v2={:?}", &v2);
}
println!("v1={:?}", &v1);

v2=[22, 44, 66]
v1=[22, 44, 66]


<div style="text-align: center">
<img alt="A reference is a non-owning pointer" src="images/trpl04-05.svg" style="display: inline; width: 16em;">
</div>

For every type `T`, variables of type `&T` are **references** to a `T`.

* Physically, references are pointers.
* Conceptually, references do *not own* the data they point to.
* Freeing the reference does not lead to freeing the pointed data.
* Several references to the same data can *co-exist*.

In [8]:
let mut v1: Vec<i32> = vec![22, 44, 66];
{
    let v2 = &v1;
    let v3 = &v1;
    println!("v1={:?} v2={:?} v3={:?}", &v1, &v2, &v3);
}
println!("v1={:?}", &v1);

v1=[22, 44, 66] v2=[22, 44, 66] v3=[22, 44, 66]
v1=[22, 44, 66]


NB : references of type `&T` are immutable.

In [9]:
let mut v1: Vec<i32> = vec![22];
v1.push(44);
{
    let v2 = &v1;
    //v2.push(66); // DOES NOT COMPILE
    println!("v2={:?}", &v2);
}
v1.push(88);
println!("v1={:?}", &v1);

v2=[22, 44]
v1=[22, 44, 88]


In order to make the previous example to work,
we need a **mutable reference**,
of type `&mut T`.

In [10]:
let mut v1: Vec<i32> = vec![22];
v1.push(44);
{
    let v2 = &mut v1;
    v2.push(66);
    println!("v2={:?}", &v2);
}
v1.push(88);
println!("v1={:?}", &v1);

v2=[22, 44, 66]
v1=[22, 44, 66, 88]


### Golden rules for references

We can not have at the same time

* several mutable references, nor
* one mutable reference and one (or several) immutable reference(s).

because we need to avoid:

* two parts of the code to modify the data simultaneously, and
* one part of the code to read data that is being modified.

This is checked by the **borrow checker** (a part of the compiler).

In [11]:
let mut v1: Vec<i32> = vec![];
v1.push(22);
{
    let v2 = &mut v1;
    v2.push(44);
    //v1.push(66);  // DOES NOT COMPILE
    //let v3 = &v1; // DOES NOT CMPILE
    println!("v2={:?}", &v2);
}
v1.push(88);
println!("v1={:?}", &v1);

v2=[22, 44]
v1=[22, 44, 88]


### References do not behave like pointers

(even though internally, they are pointers)


In [12]:
let v = vec![42, 42];
let test = (&v[0] == &v[1]);
test

true

* The "value" of a reference is **not** the memory address of the data, it is the data.

  * Better think of `&T` as "a `T` borrowed from someone else" than as "a pointer to `T`".
  * `T` and `&T` are still distinct types.
  
* In Rust, a reference can **never** be a null pointer. 🦀

## How to chose the type of function parameters

In [13]:
// my function only need to access the data: &T
fn count_zeros(v: &Vec<i32>) -> usize {
    let mut c = 0;
    for i in v {
        if *i == 0 {
            c += 1;
        }
    }
    c
}
// NB: this implementation is not idiomatic

In [14]:
// my function needs to modify the data: &mut T
fn add_one(v: &mut Vec<i32>) {
    for i in v {
        *i += 1;
    }
}

In [15]:
// my function "consumes" the data (rare) / the data is `Copy`: T
fn fact(n: usize) -> usize {
    if n <= 1 { 1 } else { n*fact(n-1) }
}

## Lifetime

The borrow checker ensures that no borrowed data is needed for a longer time than the owner of the data.

(because when the owner "dies", the data is freed).


In [16]:
// this code does not compile
{
    let mut borrower: Vec<&i32> = Vec::new();
    {
        let owner = 42;
        borrower.push(&owner);
        println!("{:?}", &borrower);
    }
    // here, 'owner' does not exist anymore (out of scope)
    println!("{:?}", &borrower);
}

Error: `owner` does not live long enough

In some situations, the compiler needs explicit hints about the respective lifetimes of different referenves
("who lifes as long as who?").

In [17]:
fn borrow_concat(v1: &Vec<i32>, v2: &Vec<i32>) -> Vec<&i32> {
    let mut v3 = Vec::with_capacity(v1.len() + v2.len());
    for i in 0..v1.len() {
        v3.push(&v1[i]);
    }
    for i in 0..v2.len() {
        v3.push(&v2[i]);
    }
    v3
}
// NB: this implementation is not idiomatic

Error: missing lifetime specifier

Solution du problème précédent : on introduit des identifiants de *lifetime*.

(En l'occurrence, un seul, arbitrairement nommé `'a`,
 pour indiquer que les trois types de références doivent vivre aussi longtemps les uns que les autres.)

In [18]:
fn borrow_concat<'a>(v1: &'a Vec<i32>, v2: &'a Vec<i32>) -> Vec<&'a i32> {
    // indicates that v1 and v2 must live as long as
    // the elements of the returned Vec
    let mut v3 = Vec::with_capacity(v1.len() + v2.len());
    for i in 0..v1.len() {
        v3.push(&v1[i]);
    }
    for i in 0..v2.len() {
        v3.push(&v2[i]);
    }
    v3
}
// NB: this implementation is not idiomatic

Rust has a number of default rules for lifetimes,
so that most of the time, it is not require to specify them.

E.g. when there is only one input reference and one output reference,
they are assume to have the same lifetime.

In [19]:
fn borrow_items(v1: &Vec<i32>) -> Vec<&i32> {
// equivalent to
// fn borrow_items<'a>(v1: &'a Vec<i32>) -> Vec<&'a i32> {
    let mut v2 = Vec::with_capacity(v1.len());
    for i in 0..v1.len() {
        v2.push(&v1[i]);
    }
    v2
}
// NB: this implementation is not idiomatic